In [10]:
import os
import requests
import json
import pandas as pd

In [11]:
url = "https://api.nasa.gov/planetary/apod" # the apod url

In [12]:
# parameters for the API
params = {
    'start_date': '',
    'end_date': '',
    'api_key': os.getenv('NASA_API_KEY')
}

In [13]:
# gets data from the url provided
response = requests.get(url+params=params)
response.status_code

In [15]:
# parse the JSON-formatted strin into a Python dictionary
data = json.loads(response.text)

In [16]:
# gotten from url
data

{'date': '2024-03-29',
 'explanation': "Looping through the Jovian system in the late 1990s, the Galileo spacecraft recorded stunning views of Europa and uncovered evidence that the moon's icy surface likely hides a deep, global ocean. Galileo's Europa image data has been remastered here, with improved calibrations to produce a color image approximating what the human eye might see. Europa's long curving fractures hint at the subsurface liquid water.  The tidal flexing the large moon experiences in its elliptical orbit around Jupiter supplies the energy to keep the ocean liquid. But more tantalizing is the possibility that even in the absence of sunlight that process could also supply the energy to support life, making Europa one of the best places to look for life beyond Earth. The Juno spacecraft currently in Jovian orbit has also made repeated flybys of the water world, returning images along with data exploring Europa's habitability. This October will see the launch of the NASA's E

## **NASA API USING THE START AND END DATE ON ANOTHER URL**

In [17]:
# working on another url
new_url = 'https://api.nasa.gov/neo/rest/v1/feed'

In [18]:
# parameters for new url
parameter = {
    'api_key': os.getenv('NASA_API_KEY'),
    'start_date': '2024-02-04',
    'end_date': '2024-02-09'
}

In [20]:
# used the requests library to get the data from the new url
new_response = requests.get(new_url, params=parameter)
new_response.status_code

200

In [21]:
# parse the JSON-formatted strin into a Python dictionary
new_data = json.loads(new_response.text)
new_data

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-02-09&end_date=2024-02-14&detailed=false&api_key=Fr92ikwh5AB0Q9ctJfNpIuIJrNTrUZuXxiFFmCWf',
  'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-30&end_date=2024-02-04&detailed=false&api_key=Fr92ikwh5AB0Q9ctJfNpIuIJrNTrUZuXxiFFmCWf',
  'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-02-04&end_date=2024-02-09&detailed=false&api_key=Fr92ikwh5AB0Q9ctJfNpIuIJrNTrUZuXxiFFmCWf'},
 'element_count': 120,
 'near_earth_objects': {'2024-02-05': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2347813?api_key=Fr92ikwh5AB0Q9ctJfNpIuIJrNTrUZuXxiFFmCWf'},
    'id': '2347813',
    'neo_reference_id': '2347813',
    'name': '347813 (2002 NP1)',
    'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2347813',
    'absolute_magnitude_h': 17.66,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.7808272775,
      'estimated_diameter_max': 1.7459828712},
 

In [23]:
# the data gotten here appears to be in dictionary
new_data.keys()

dict_keys(['links', 'element_count', 'near_earth_objects'])

In [24]:
new_data['near_earth_objects']

{'2024-02-05': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2347813?api_key=Fr92ikwh5AB0Q9ctJfNpIuIJrNTrUZuXxiFFmCWf'},
   'id': '2347813',
   'neo_reference_id': '2347813',
   'name': '347813 (2002 NP1)',
   'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2347813',
   'absolute_magnitude_h': 17.66,
   'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.7808272775,
     'estimated_diameter_max': 1.7459828712},
    'meters': {'estimated_diameter_min': 780.8272774914,
     'estimated_diameter_max': 1745.9828711568},
    'miles': {'estimated_diameter_min': 0.4851834262,
     'estimated_diameter_max': 1.0849031226},
    'feet': {'estimated_diameter_min': 2561.7693650848,
     'estimated_diameter_max': 5728.2904430061}},
   'is_potentially_hazardous_asteroid': False,
   'close_approach_data': [{'close_approach_date': '2024-02-05',
     'close_approach_date_full': '2024-Feb-05 03:32',
     'epoch_date_close_approach': 1707103920000,
     'rel

## **TASK**

Do the necessary data pre-processing tasks on the previous result in order to get a clean dataframe with the following columns:
- Asteroid ID
- Asteroid name
- The minimal estimated diameter in Kilometre
- Absolute magnitude
- Relative velocity(km/s)

In [25]:
# asteroid id
new_data['near_earth_objects']['2024-02-05'][0]['id']

'2347813'

In [26]:
# asteroid name
new_data['near_earth_objects']['2024-02-05'][0]['name']

'347813 (2002 NP1)'

In [27]:
# minimal estimated diameter in KM
new_data['near_earth_objects']['2024-02-05'][0]['estimated_diameter']['kilometers']['estimated_diameter_min']

0.7808272775

In [28]:
# absolute magnitude relative velocity(km/s)
new_data['near_earth_objects']['2024-02-05'][0]['absolute_magnitude_h']

17.66

In [29]:
# relative velocity(km/s)
new_data['near_earth_objects']['2024-02-05'][2]['close_approach_data'][0]['relative_velocity']['kilometers_per_second']

'8.2382303218'

## **PUT DATA INTO A DATAFRAME**

In [30]:
# create a DataFrame
df = pd.DataFrame() 
# loop through each day
for days in new_data['near_earth_objects']:
    # save the value for that day in a variable 'daily_near_earth_object'
    daily_near_earth_object = new_data['near_earth_objects'][days]
    # loop through the 'daily_near_earth_object'
    for near_earth_object in daily_near_earth_object:
        # create a temporary DataFrame 
        temp = pd.DataFrame(
            {
                'Asteroid_ID': [near_earth_object['id']],
                'Asteroid_name': [near_earth_object['name']],
                'Minimal_estimated_diameter(Km)': [near_earth_object['estimated_diameter']['kilometers']['estimated_diameter_min']],
                'Absolute_magnitude': [near_earth_object['absolute_magnitude_h']],
                'Relative_velocity(Km/s)': [near_earth_object['close_approach_data'][0]['relative_velocity']['kilometers_per_second']]
            }
        )
        # combine the temporary DataFrames
        df = pd.concat([df, temp], ignore_index=True)
df.head()

,Asteroid_ID,Asteroid_name,Minimal_estimated_diameter(Km),Absolute_magnitude,Relative_velocity(Km/s)
0,2347813,347813 (2002 NP1),0.780827,17.66,13.310416392
1,2444627,444627 (2006 WU),0.411675,19.05,1.9982604604
2,2451370,451370 (2011 AK5),0.134448,21.48,8.2382303218
3,3440771,(2008 WZ94),0.235806,20.26,16.1492348048
4,3557844,(2011 DW),0.076658,22.70,14.872591907


In [31]:
df.shape

(120, 5)